In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install lightgbm

In [ ]:
train_data = pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv" ) # reading the train data to a data frame 
test_data = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv' ) # reading the test data into a data frame 
sample_submission = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv') # reading the test data into a data frame
print(" data imported keep going....")

In [ ]:
for c in train_data.columns:
    col_type = train_data[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        train_data[c] = train_data[c].astype('category')

for c in test_data.columns:
    col_type = test_data[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        test_data[c] = test_data[c].astype('category')

### Making a copy of Datetime column in new column because datetime now is categorical format the way LIGHTGBM accepts it

In [ ]:
train_data['date_time_new'] = train_data['date_time']
test_data['date_time_new'] = test_data['date_time']

In [ ]:
# coverting category columns to datetime
train_data['date_time'] = pd.to_datetime(train_data['date_time'])
test_data['date_time'] = pd.to_datetime(test_data['date_time'])

### The Features I have commented have dropped our score found that they are not useful from [here](https://www.kaggle.com/c/tabular-playground-series-jul-2021/discussion/250931) 

In [ ]:

train_data["quarter"] = train_data["date_time"].dt.quarter
train_data['month'] = train_data['date_time'].dt.month
# train_data["is_winter"] = train_data["month"].isin([1, 2, 12])
# train_data["is_sprint"] = train_data["month"].isin([3, 4, 5])
# train_data["is_summer"] = train_data["month"].isin([6, 7, 8])
# train_data["is_autumn"] = train_data["month"].isin([9, 10, 11])
train_data['hour'] = train_data['date_time'].dt.hour
train_data["working_hours"] =  train_data["hour"].isin(np.arange(8, 21, 1)).astype("int")
train_data["is_weekend"] = (train_data["date_time"].dt.dayofweek >= 5).astype("int")

train_data['SMC'] = (train_data['absolute_humidity'] * 100) / train_data['relative_humidity']
train_data['Dew_Point'] = 243.12*(np.log(train_data['relative_humidity'] * 0.01) + 
                          (17.62 * train_data['deg_C'])/(243.12+train_data['deg_C']))/(17.62-(np.log(train_data['relative_humidity'] * 0.01)+17.62*train_data['deg_C']/(243.12+train_data['deg_C'])))

# lag features
train_data["dt-6"] = train_data["deg_C"] - train_data["deg_C"].shift(periods=6, fill_value=0)
train_data["dt-3"] = train_data["deg_C"] - train_data["deg_C"].shift(periods=3, fill_value=0)
train_data["dt-2"] = train_data["deg_C"] - train_data["deg_C"].shift(periods=2, fill_value=0)

train_data["abshum-6"] = train_data["absolute_humidity"] - train_data["absolute_humidity"].shift(periods=6, fill_value=0)
train_data["abshum-3"] = train_data["absolute_humidity"] - train_data["absolute_humidity"].shift(periods=3, fill_value=0)
###


# convert datetime to seconds 
train_data['time'] = train_data['date_time'].astype('datetime64[ns]').astype(np.int64)//10**9

# day of week
train_data['day_of_week']=train_data['date_time'].dt.dayofweek




In [ ]:
test_data["quarter"] = test_data["date_time"].dt.quarter
test_data['month'] = test_data['date_time'].dt.month
# test_data["is_winter"] = test_data["month"].isin([1, 2, 12])
# test_data["is_sprint"] = test_data["month"].isin([3, 4, 5])
# test_data["is_summer"] = test_data["month"].isin([6, 7, 8])
# test_data["is_autumn"] = test_data["month"].isin([9, 10, 11])
test_data['hour'] = test_data['date_time'].dt.hour
test_data["working_hours"] =  test_data["hour"].isin(np.arange(8, 21, 1)).astype("int")
test_data["is_weekend"] = (test_data["date_time"].dt.dayofweek >= 5).astype("int")
test_data['SMC'] = (test_data['absolute_humidity'] * 100) / test_data['relative_humidity']
test_data['Dew_Point'] = 243.12*(np.log(test_data['relative_humidity'] * 0.01) + 
                          (17.62 * test_data['deg_C'])/(243.12+test_data['deg_C']))/(17.62-(np.log(test_data['relative_humidity'] * 0.01)+17.62*test_data['deg_C']/(243.12+test_data['deg_C'])))

# lag features
test_data["dt-6"] = test_data["deg_C"] - test_data["deg_C"].shift(periods=6, fill_value=0)
test_data["dt-3"] = test_data["deg_C"] - test_data["deg_C"].shift(periods=3, fill_value=0)
test_data["dt-2"] = test_data["deg_C"] - test_data["deg_C"].shift(periods=2, fill_value=0)

test_data["abshum-6"] = test_data["absolute_humidity"] - test_data["absolute_humidity"].shift(periods=6, fill_value=0)
test_data["abshum-3"] = test_data["absolute_humidity"] - test_data["absolute_humidity"].shift(periods=3, fill_value=0)
###############

# convert datetime to seconds 
test_data['time'] = test_data['date_time'].astype('datetime64[ns]').astype(np.int64)//10**9

# day of week
test_data['day_of_week']=test_data['date_time'].dt.dayofweek



##### **We again copied datetime to date_time_new because LightGBm accepts Categorical columns in category type**
#####  **and then dropped date_time** columns

In [ ]:

train_data.drop(['date_time'],axis=1,inplace =True)
test_data.drop(['date_time'],axis=1,inplace =True)

In [ ]:
print(train_data.shape)
print(test_data.shape)

targets = ['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']

y = train_data.loc[:,targets]

train_data.drop(labels = targets,inplace=True,axis=1)

In [ ]:
y1 = np.array(y.loc[:, list(y.columns)[0] ] )# first column for prediction

y2 = np.array(y.loc[:, list(y.columns)[1] ]) # second column for prediction

y3 = np.array(y.loc[:, list(y.columns)[2] ]) # third column for prediction

In [ ]:
cat_cols = train_data.select_dtypes('category').columns.tolist()
cat_cols

In [ ]:
import math
from tqdm.notebook import tqdm_notebook
from sklearn.metrics import mean_squared_log_error as msle

In [ ]:
iterations =  50
_pred_1 = 0
rmsle_1 = []

In [ ]:
from sklearn.model_selection import StratifiedKFold ,KFold
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

for i in tqdm_notebook(range(iterations)):
    random_state = np.random.randint(0,2000)
    X_train, X_valid, y_train, y_valid = train_test_split(train_data, y1 , test_size=0.2, random_state=random_state)
    lgbm = LGBMRegressor(random_state=random_state)
    lgbm.fit(X_train,y_train)
    valid_preds = lgbm.predict(X_valid)
    
    # MSLE cannot predict values if predicted values is less than zero just making sure we dont get neagitives
    if np.any(valid_preds<0):
        valid_preds = np.array([0 if i < 0 else i for i in valid_preds])
    _pred_1 += lgbm.predict(test_data)/iterations
    rmsle_error = math.sqrt(msle(y_valid,valid_preds))
    rmsle_1.append(rmsle_error)
    print(f'Iteration {i} : Random State = {random_state}, RMSLE Loss = {rmsle_error}')
print(f'Average RMSLE Loss = {sum(rmsle_1)/len(rmsle_1)}')

In [ ]:
iterations =  50 
_pred_2 = 0
rmsle_2 = []

for i in tqdm_notebook(range(iterations)):
    random_state = np.random.randint(0,2000)
    X_train, X_valid, y_train, y_valid = train_test_split(train_data, y2 , test_size=0.2, random_state=random_state)
    lgbm = LGBMRegressor(random_state=random_state)
    lgbm.fit(X_train,y_train)
    valid_preds = lgbm.predict(X_valid)
    if np.any(valid_preds<0):
        valid_preds = np.array([0 if i < 0 else i for i in valid_preds])
    _pred_2 += lgbm.predict(test_data)/iterations
    rmsle_error = math.sqrt(msle(y_valid,valid_preds))
    rmsle_2.append(rmsle_error)
    print(f'Iteration {i} : Random State = {random_state}, RMSLE Loss = {rmsle_error}')
print(f'Average RMSLE Loss = {sum(rmsle_2)/len(rmsle_2)}')

In [ ]:
iterations =50
_pred_3 = 0
rmsle_3 = []

for i in tqdm_notebook(range(iterations)):
    random_state = np.random.randint(0,2000)
    X_train, X_valid, y_train, y_valid = train_test_split(train_data, y3 , test_size=0.2, random_state=random_state)
    lgbm = LGBMRegressor(random_state=random_state)
    lgbm.fit(X_train,y_train)
    valid_preds = lgbm.predict(X_valid)
    if np.any(valid_preds<0):
        valid_preds = np.array([0 if i < 0 else i for i in valid_preds])
    _pred_3 += lgbm.predict(test_data)/iterations
    rmsle_error = math.sqrt(msle(y_valid,valid_preds))
    rmsle_3.append(rmsle_error)
    print(f'Iteration {i} : Random State = {random_state}, RMSLE Loss = {rmsle_error}')
print(f'Average RMSLE Loss = {sum(rmsle_3)/len(rmsle_3)}')

In [ ]:
test_sub = np.vstack([_pred_1,_pred_2,_pred_3]).T

In [ ]:
test_sub.shape

In [ ]:
sample_submission[sample_submission.columns[1:]] = test_sub
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission.csv',index=False)